# 联通用户流失名单预测

## 项目背景
国内通信市场逐渐的成熟，三大运营商营销模式业务日益趋同，竞争压力比较大，高新增用户已经成为过去式，用户的流失率提高已经成为普遍现象。在通信市场上，联通公司面临着移动和电信的强大挑战。如何提高用户的满意度，降低流失率成为主要问题，有效的发展客户，进而提高收入，成为了联通公司运营管理者的主要问题。

<img src='./运营商数据.png'>

## 需求拆解
用户流失的原因：信号不稳定，网速慢，价格不划算  
交付：每周提交一次用户流失概率比较大的人员名单

## 收集数据
价格不划算： 套餐价格（对比竞品）  
超出套餐部分的通话费用比较贵：对即将超出套餐的进行提醒，并合理推荐套餐  
超出套餐部分的流量费用比较贵：对即将超出套餐的进行提醒，并合理推荐套餐  

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('./联通用户流失数据/CustomerSurvival.xlsx')

data.head()


,ID,套餐金额,额外通话时长,额外流量,改变行为,服务合约,关联购买,集团用户,使用月数,流失用户
0,1,1,792.833333,-10.450067,0,0,0,0,25,0
1,2,1,121.666667,-21.141117,0,0,0,0,25,0
2,3,1,-30.000000,-25.655273,0,0,0,0,2,1
3,4,1,241.500000,-288.341254,0,1,0,1,25,0
4,5,1,1629.666667,-23.655505,0,0,0,1,25,0


## 数据处理

### 缺失值

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975 entries, 0 to 4974
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      4975 non-null   int64  
 1   套餐金额    4975 non-null   int64  
 2   额外通话时长  4975 non-null   float64
 3   额外流量    4975 non-null   float64
 4   改变行为    4975 non-null   int64  
 5   服务合约    4975 non-null   int64  
 6   关联购买    4975 non-null   int64  
 7   集团用户    4975 non-null   int64  
 8   使用月数    4975 non-null   int64  
 9   流失用户    4975 non-null   int64  
dtypes: float64(2), int64(8)
memory usage: 388.8 KB


### 异常值

In [4]:
data['额外通话时长'].quantile([0, 0.25, 0.5, 0.75, 1])

0.00   -2828.333333
0.25    -126.666667
0.50      13.500000
0.75     338.658333
1.00    4314.000000
Name: 额外通话时长, dtype: float64

In [5]:
(data['额外通话时长'] > 339).sum()

1241

In [6]:
# 在业务指标值中，认定超过3000，或者剩余3000为异常值
data = data[data['额外通话时长'] < 3000]
data = data[data['额外通话时长'] > -3000]

In [7]:
# 额外流量
# Q1 = data['额外流量'].quantile(0.25)
# Q3 = data['额外流量'].quantile(0.75)
# IQR = Q3 - Q1
# max_val = Q3 + 1.5*IQR
# min_val = Q1 - 1.5*IQR
# max_val,min_val

In [8]:
# data['额外流量'].max()
# data['额外流量'].min()

## 特征工程

In [9]:
# 删除ID列
del data['ID']

In [10]:
# 连续数据，离散化
extra_time_cut = [[-3000, -1000, 0, 1000, 3000], [2, 4, 3, 1]]

data['额外通话时长'] = pd.cut(data['额外通话时长'],
                              bins=[-3000, -1000, 0, 1000, 3000],
                              labels=[2, 4, 3, 1])

data['额外流量'] = np.where(data['额外流量'] > 0, 2, 1)


## 模型选择

In [11]:
# 单科决策树，Adaboost，GBDT，随机森林  （谁好选谁）
import sklearn.model_selection as ms  #模型选择
import sklearn.tree as st  #决策树
import sklearn.ensemble as se  #集成学习
import sklearn.metrics as sm  #评估模块

In [12]:
# 整理输入和输出
x = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [13]:
train_x, test_x, train_y, test_y = ms.train_test_split(x, y,
                                                       test_size=0.1,
                                                       random_state=7,
                                                       stratify=y)

In [14]:
def select_model(name, model):
    print('--------', name, '----------')
    model.fit(train_x, train_y)
    pred_test_y = model.predict(test_x)
    print(sm.classification_report(test_y, pred_test_y))


model_dict = {'单颗决策树': st.DecisionTreeClassifier(),
              'Adaboost': se.AdaBoostClassifier(st.DecisionTreeClassifier(),
                                                n_estimators=100),
              'GBDT': se.GradientBoostingClassifier(n_estimators=100),
              '随机森林': se.RandomForestClassifier(n_estimators=100)}

for name, obj in model_dict.items():
    select_model(name, obj)


-------- 单颗决策树 ----------
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       107
           1       0.98      0.99      0.98       383

    accuracy                           0.97       490
   macro avg       0.96      0.95      0.96       490
weighted avg       0.97      0.97      0.97       490

-------- Adaboost ----------
              precision    recall  f1-score   support

           0       0.96      0.90      0.93       107
           1       0.97      0.99      0.98       383

    accuracy                           0.97       490
   macro avg       0.97      0.94      0.95       490
weighted avg       0.97      0.97      0.97       490

-------- GBDT ----------
              precision    recall  f1-score   support

           0       0.98      0.90      0.94       107
           1       0.97      0.99      0.98       383

    accuracy                           0.97       490
   macro avg       0.98      0.95      0.96     

## 模型的优化

In [15]:
sub_model = st.DecisionTreeClassifier()
params = {'criterion': ['gini', 'entropy'],
          'max_depth': np.arange(2, 9),
          'min_samples_split': np.arange(2, 21),
          'min_samples_leaf': np.arange(1, 11)}

sub_GS = ms.GridSearchCV(sub_model, params, cv=3)
sub_GS.fit(x, y)
1

1

In [16]:
main_model = se.AdaBoostClassifier(sub_GS.best_estimator_)

params = {'n_estimators': np.arange(20, 201, 10)}

main_GS = ms.GridSearchCV(main_model, params, cv=3)
main_GS.fit(x, y)

best_model = main_GS.best_estimator_


In [17]:
pred_best_y = best_model.predict(test_x)
print(sm.classification_report(test_y, pred_best_y))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96       107
           1       0.98      0.99      0.99       383

    accuracy                           0.98       490
   macro avg       0.98      0.97      0.97       490
weighted avg       0.98      0.98      0.98       490



## 保存模型

In [18]:
import pickle

In [19]:
dict_info = {'数据结构': data.columns[:-1],
             '数据转换': {'额外通话时长': extra_time_cut,
                          '额外流量': {'条件>': 0, 'True': 2, 'False': 1}},
             '模型': best_model}

In [20]:
with open('联通用户流失预测模型.pickle', 'wb') as f:
    pickle.dump(dict_info, f)
print('模型保存成功')

模型保存成功


## 加载模型

In [21]:
with open('./联通用户流失预测模型.pickle', 'rb') as f:
    obj = pickle.load(f)

In [22]:
obj['数据结构']

Index(['套餐金额', '额外通话时长', '额外流量', '改变行为', '服务合约', '关联购买', '集团用户', '使用月数'], dtype='object')

In [23]:
need_data = [[1, 1000, 500, 0, 0, 1, 0, 25],
             [2, 0, 0, 1, 1, 2, 1, 25],
             [1, -500, -500, 0, 1, 0, 1, 13]]
need_data = pd.DataFrame(need_data, columns=obj['数据结构'])

In [24]:
need_data['额外通话时长'] = pd.cut(need_data['额外通话时长'], bins=obj['数据转换']['额外通话时长'][0],
                                   labels=obj['数据转换']['额外通话时长'][1])

In [25]:
need_data['额外流量'] = np.where(need_data['额外流量'] > obj['数据转换']['额外流量']['条件>'],
                                 obj['数据转换']['额外流量']['True'],
                                 obj['数据转换']['额外流量']['False'])

In [30]:
need_data


,套餐金额,额外通话时长,额外流量,改变行为,服务合约,关联购买,集团用户,使用月数
0,1,3,2,0,0,1,0,25
1,2,4,1,1,1,2,1,25
2,1,4,1,0,1,0,1,13


In [32]:
obj['模型'].predict_proba(need_data)  #将训练数据丢入模型可以得到置信概率

array([[9.99999976e-01, 2.43147961e-08],
       [9.99999990e-01, 1.02756894e-08],
       [3.24878628e-06, 9.99996751e-01]])

In [27]:
#need_data['流失概率'] = obj['模型'].predict_proba(need_data)[:,-1]

In [28]:
# 交付流失概率前50名的数据
#res = need_data.sort_values(by='流失概率',ascending=False).head(50)

## 交付内容

In [29]:
#res.to_csv('流失预测结果.csv')